## Dependencies

In [3]:
# AWS Imports
import boto3
from botocore.client import ClientError
import sagemaker
from pyathena import connect
import awswrangler as wr

# Data Transformation Imports
import pandas as pd
from io import StringIO

# Misc Imports
from IPython.display import display, HTML

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## Sagemaker Details and Variable Init

In [6]:
# Create a SageMaker session object, which is used to manage interactions with SageMaker resources.
sess = sagemaker.Session()

# Retrieve the default Amazon S3 bucket associated with the SageMaker session.
bucket = sess.default_bucket()

# Get the IAM role associated with the current SageMaker notebook or environment.
role = sagemaker.get_execution_role()

# Get the AWS region name for the current session.
region = boto3.Session().region_name

# Retrieve the AWS account ID of the caller using the Security Token Service (STS) client.
account_id = boto3.client("sts").get_caller_identity().get("Account")

# Create a Boto3 client for the SageMaker service, specifying the AWS region.
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [41]:
# Initializing variables for reproducibility
DATA_PATH = f"s3://{bucket}/aai-540-group-3-final-project/data/"
DATABASE = "retainAI"
PROD_DIR = f"s3://{bucket}/athena/prod"
STAGE_DIR = f"s3://{bucket}/athena/staging"
EMPLOYEE_TABLE = "employee_table"

# Making sure all variables are correct

print(f"Training and Testing Path: {DATA_PATH}")
print(f"Production and Staging Database Directories: {PROD_DIR},{STAGE_DIR}")
print(f"Database Name, Training Table and Testing Table: {DATABASE}, {EMPLOYEE_TABLE}")

Training and Testing Path: s3://sagemaker-us-east-1-203012117619/aai-540-group-3-final-project/data/
Production and Staging Database Directories: s3://sagemaker-us-east-1-203012117619/athena/prod,s3://sagemaker-us-east-1-203012117619/athena/staging
Database Name, Training Table and Testing Table: retainAI, employee_table


## Creating Athena Schema

In [29]:
# Establish a connection to the AWS Athena service, specifying the region and an S3 staging directory
# where query results will be stored.
conn = connect(region_name=region, s3_staging_dir=STAGE_DIR)

# Define a SQL statement to create a database in Athena if it doesn't already exist.
# The database name is dynamically determined by the variable `DATABASE`.
db_create_statement = f"CREATE DATABASE IF NOT EXISTS {DATABASE}"

# Execute the SQL statement using the established connection and Pandas, 
# which sends the query to Athena and ensures the database is created.
pd.read_sql(db_create_statement, conn)

/tmp/ipykernel_2563/693278258.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(db_create_statement, conn)


""


In [30]:
# Define a SQL statement to list all existing databases in the AWS Athena service.
show_db_statement = "SHOW DATABASES"

# Execute the SQL query using the established connection, and store the results in a Pandas DataFrame.
df_show = pd.read_sql(show_db_statement, conn)

# Display the first 5 rows of the DataFrame to view the list of databases.
df_show.head(5)

/tmp/ipykernel_2563/1619648943.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(show_db_statement, conn)


,database_name
0,aws
1,default
2,dsoaws
3,retainai


## Registering Data with Athena

### Creating Employee Data Table

In [42]:
CREATE_STATEMENT = """
CREATE EXTERNAL TABLE IF NOT EXISTS {}.{} (
    employee_id INT,
    age INT,
    gender STRING,
    years_at_company INT,
    job_role STRING,
    monthly_income INT,
    work_life_balance STRING,
    job_satisfaction STRING,
    performance_rating STRING,
    number_of_promotions INT,
    distance_from_home INT,
    education_level STRING,
    marital_status STRING,
    number_of_dependents INT,
    job_level STRING,
    company_size STRING,
    company_tenure INT,
    remote_work STRING,
    leadership_opportunities STRING,
    innovation_opportunities STRING,
    company_reputation STRING,
    employee_recognition STRING,
    attrition STRING
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\\n'
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')
""".format(DATABASE, EMPLOYEE_TABLE, DATA_PATH)

# Ensuring everything is formatted correctly
print(CREATE_STATEMENT)


CREATE EXTERNAL TABLE IF NOT EXISTS retainAI.employee_table (
    employee_id INT,
    age INT,
    gender STRING,
    years_at_company INT,
    job_role STRING,
    monthly_income INT,
    work_life_balance STRING,
    job_satisfaction STRING,
    performance_rating STRING,
    number_of_promotions INT,
    distance_from_home INT,
    education_level STRING,
    marital_status STRING,
    number_of_dependents INT,
    job_level STRING,
    company_size STRING,
    company_tenure INT,
    remote_work STRING,
    leadership_opportunities STRING,
    innovation_opportunities STRING,
    company_reputation STRING,
    employee_recognition STRING,
    attrition STRING
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
LOCATION 's3://sagemaker-us-east-1-203012117619/aai-540-group-3-final-project/data/'
TBLPROPERTIES ('skip.header.line.count'='1')



In [37]:
# Execute the SQL query defined in CREATE_STATEMENT to create the employee data table in Athena. 
pd.read_sql(CREATE_STATEMENT, conn)

/tmp/ipykernel_2563/3582723373.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(TRAIN_CREATE_STATEMENT, conn)


""


In [32]:
TEST_CREATE_STATEMENT = """
CREATE EXTERNAL TABLE IF NOT EXISTS {}.{} (
    employee_id INT,
    age INT,
    gender STRING,
    years_at_company INT,
    job_role STRING,
    monthly_income INT,
    work_life_balance STRING,
    job_satisfaction STRING,
    performance_rating STRING,
    number_of_promotions INT,
    distance_from_home INT,
    education_level STRING,
    marital_status STRING,
    number_of_dependents INT,
    job_level STRING,
    company_size STRING,
    company_tenure INT,
    remote_work STRING,
    leadership_opportunities STRING,
    innovation_opportunities STRING,
    company_reputation STRING,
    employee_recognition STRING,
    attrition STRING
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\\n'
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')
""".format(DATABASE, TESTING_TABLE, BUCKET_PATH_TEST)

# Ensuring everything is formatted correctly
print(TEST_CREATE_STATEMENT)


CREATE EXTERNAL TABLE IF NOT EXISTS retainAI.testing_table (
    employee_id INT,
    age INT,
    gender STRING,
    years_at_company INT,
    job_role STRING,
    monthly_income INT,
    work_life_balance STRING,
    job_satisfaction STRING,
    performance_rating STRING,
    number_of_promotions INT,
    distance_from_home INT,
    education_level STRING,
    marital_status STRING,
    number_of_dependents INT,
    job_level STRING,
    company_size STRING,
    company_tenure INT,
    remote_work STRING,
    leadership_opportunities STRING,
    innovation_opportunities STRING,
    company_reputation STRING,
    employee_recognition STRING,
    attrition STRING
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
LOCATION 's3://sagemaker-us-east-1-203012117619/aai-540-group-3-final-project/data/test.csv'
TBLPROPERTIES ('skip.header.line.count'='1')



### Check Tables and Ensure Correctness

If everything runs correctly, you should see tab_name = employee_table

In [43]:
check_table_statement = "SHOW TABLES in {}".format(DATABASE)

df_show = pd.read_sql(check_table_statement, conn)
df_show

/tmp/ipykernel_2563/457795480.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(check_table_statement, conn)


,tab_name
0,employee_table


In [44]:
# If everything runs as expected, you should get a return value of 74,498 instances in our table
count_records = f"""SELECT COUNT(*) FROM {DATABASE}.{EMPLOYEE_TABLE}"""
count_show = pd.read_sql(count_records,conn)
count_show

/tmp/ipykernel_2563/2187136526.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  count_show = pd.read_sql(count_records,conn)


,_col0
0,74498


## Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}